## Hackin 2

- Ok ok ok ok ok ok
- I dont want to overcomplicate things here
- I think this visual could be dope, and all I need at this phase is some proof of concept that I can see some meaningful difference betwen types of examples
- I dont' have to even train, just compute grads and visualize I think, and see if my my quick/rough max-pooley appraoch can reveal some pattters here
- Claude had some good ideas for types of eamples

- Code completation vs creative writing
- Technical explanations vs emotional/narrative text
- math problems
- factual recall
- Short vs long
- repeated combinations

Ok let me make a few baby datasets -> maybe like 5-7 examples each?

In [1]:
# ! pip install transformers matplotlib tqdm huggingface_hub

In [2]:
# from huggingface_hub import login
# login()

In [3]:
import torch
from transformers import pipeline
from torch.nn import functional as F
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import os

from transformers import LlamaForCausalLM, PreTrainedTokenizerFast, LlamaConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

device='cuda'

In [4]:
# baby_dataset_so_cute=[
#     "2 + 2 = ",
#     "4 + 3 = ",
#     "2 + 9 = ",
#     "11 - 10 = ",
#     "3 - 6 = ", #Gets this one wrong
#     "3 * 6 = ",
#     "3 * 2 = ",
#     "8 / 2 = ",
#     "128 / 32 = ",
#     "The capital of France is",
    
# ]

baby_dataset_so_cute=[
    #jibberish
    "asdflkjasdf",
    "189sdfsf8sdf8sns",
    "asdfasdfuowofof",
    
    # Arithmetic (10 examples)
    "2 + 2 = ",
    "4 + 3 = ",
    "2 + 9 = ",
    "11 - 10 = ",
    "3 - 6 = ",  # Gets this one wrong
    "3 * 6 = ",
    "3 * 2 = ",
    "8 / 2 = ",
    "128 / 32 = ",
    "9 * 7 = ",
    
    # Geography facts (8 examples)
    "The capital of France is",
    "The capital of Japan is",
    "The capital of Brazil is",
    "Mount Everest is located in",
    "The Amazon River flows through",
    "The largest continent is",
    "The Pacific Ocean borders",
    "The currency of China is",
    
    # Sports facts (8 examples)
    "The Lakers play in",
    "The World Cup is held every",
    "Serena Williams plays",
    "The Super Bowl happens in",
    "A basketball team has",
    "The Olympics occur every",
    "Tiger Woods is famous for",
    "Cristiano Ronaldo plays",
    
    # Code (10 examples)
    "def add_numbers(a, b):",
    "import numpy as",
    "for i in range(",
    "if x > 0:",
    "print('Hello",
    "class Dog:",
    "return x +",
    "try:",
    "from datetime import",
    "x = [1, 2, 3,",
    
    # Creative writing (10 examples)
    "Once upon a time, in a magical forest",
    "The old wizard looked up at the stars and",
    "She opened the mysterious door and found",
    "The dragon's eyes glowed softly as",
    "In the bustling marketplace, children",
    "As the sun set behind the mountains,",
    "The little robot whirred to life and",
    "Deep in the ocean, a mermaid",
    "The spaceship landed with a gentle",
    "Through the mist came the sound of",

    # Logical reasoning/word relationships (8 examples)
    "If all cats are animals, and Fluffy is a cat, then Fluffy is",
    "Apple is to fruit as carrot is to",
    "Hot is the opposite of",
    "Bird is to fly as fish is to",
    "Red, blue, green, yellow,",  # sequence completion
    "Monday, Tuesday, Wednesday,",
    "January comes before",
    "Bigger than an elephant, smaller than a",

    #Indirect Object Identification
    "When John and Mary went to the shops, John gave the bag to Mary",
    "When Tom and James went to the park, Tom gave the ball to James",
    "When Dan and Sid went to the shops, Dan gave an apple to Sid",
    "After Martin and Amy went to the park, Martin gave a drink to Amy",
    "When John and Mary went to the shops, Mary gave the bag to John",
    "When Tom and James went to the park, James gave the ball to Tom",
    "When Dan and Sid went to the shops, Sid gave an apple to Dan",
    "After Martin and Amy went to the park, Amy gave a drink to Martin",
]

In [5]:
model_id = "meta-llama/Llama-3.2-1B"

In [6]:
model = AutoModelForCausalLM.from_pretrained(model_id).to(device)

# model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.float16).to(device) #Numerical stability? Seems like this work? Saves a lot of memory!
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [7]:
save_dir='/workspace/may_12_8'
os.makedirs(save_dir, exist_ok=True)
kernel_size=64
stride=64

avg_pool = torch.nn.AvgPool2d(kernel_size=kernel_size, stride=stride)
max_pool=torch.nn.MaxPool2d(kernel_size=kernel_size, stride=stride)

tensor_names=['self_attn.q_proj.weight', 'self_attn.k_proj.weight', 'self_attn.v_proj.weight', 'self_attn.o_proj.weight', 
            'mlp.gate_proj.weight', 'mlp.up_proj.weight', 'mlp.down_proj.weight']

In [8]:
filtered_params = {name: p for name, p in model.named_parameters() if p.requires_grad}

In [9]:
global_min=0
global_max=0
for k,v in filtered_params.items(): #Hmm i guess I'm doing the global weight average not the global grad average - do we care right now?
    if v.max().item()>global_max:
        global_max=v.max().item()
    if v.min().item()<global_min:
        global_min=v.min().item()

In [10]:
lr=1e-6
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [ ]:
for i in tqdm(range(len(baby_dataset_so_cute))):
    inputs = tokenizer(baby_dataset_so_cute[i], return_tensors="pt").to(device)
    input_ids = inputs["input_ids"]

    model.train()
    optimizer.zero_grad()
    outputs = model(**inputs, labels=inputs['input_ids'])
    loss = outputs.loss #Ok not just paris loss here -> not sure how much I'm worried about that
    loss.backward()
    
    # print(loss.item())
    # optimizer.step() ##Don't actually train!

    plt.clf()
    fig=plt.figure(0,(16,9), facecolor='k')
    for layer_num in range(16):
        for tensor_index in range(len(tensor_names)):
            tensor_name=tensor_names[tensor_index]
            # w=filtered_params['model.layers.'+str(layer_num)+'.'+tensor_name].detach().cpu()
            g=filtered_params['model.layers.'+str(layer_num)+'.'+tensor_name].grad.detach().cpu()
            
            # w_pooled=max_pool(w.unsqueeze(0))
            g_pooled=max_pool(g.unsqueeze(0))
            # g_pooled=avg_pool(g.unsqueeze(0))
    
            fig.add_subplot(7, 16, tensor_index*16+layer_num+1)    
            plt.imshow(g_pooled[0], vmin=-0.12, vmax=0.12)
            plt.axis('off')

    # plt.axis('on')
    fig.suptitle(baby_dataset_so_cute[i], color='w')
    plt.savefig(save_dir + '/' + 'grads_max_pooled_global_norm_' + str(i).zfill(3) +'.png', dpi=150, facecolor='k')

  2%|▏         | 1/57 [00:05<04:49,  5.18s/it]/tmp/ipykernel_4244/921253245.py:15: UserWarning: Ignoring specified arguments in this call because figure with num: 0 already exists
  fig=plt.figure(0,(16,9), facecolor='k')
 44%|████▍     | 25/57 [02:12<02:43,  5.12s/it]

---

In [ ]:
text = "The capital of France is"

inputs = tokenizer(text, return_tensors="pt").to(device)
input_ids = inputs["input_ids"]

with torch.no_grad():
    outputs = model(input_ids, labels=input_ids)

my_probs=F.softmax(outputs.logits, dim=-1)
sI=np.argsort(my_probs[0,-1, :].detach().cpu().float().numpy())[::-1]
for i in sI[:10]:
    print(i, round(my_probs[0, -1, i].item(),5), tokenizer.decode([i]))

In [ ]:
# text = "The capital of France is Berlin"
# text = "Born in Chicago, Illinois, United States and raised in a town near the city's South Side, Prevost became a friar of the Order of Saint Augustine in 1977 and was ordained priest in 1982. His service has included extensive missionary work in Peru from 1985 to 1986 and from 1988 to 1998, where he variously served as a parish pastor, diocesan official, seminary teacher, and administrator. Elected prior general of the Order of Saint Augustine from 2001 to 2013, he later returned to Peru as Bishop of Chiclayo from 2015 to 2023. In 2023, Pope Francis appointed him prefect of the Dicastery for Bishops and president of the Pontifical Commission for Latin America, and made him a cardinal the same year."
text = "2 + 2 = "

inputs = tokenizer(text, return_tensors="pt").to(device)
input_ids = inputs["input_ids"]

with torch.no_grad():
    outputs = model(input_ids, labels=input_ids)

my_probs=F.softmax(outputs.logits, dim=-1)
y_one_hot=F.one_hot(input_ids, num_classes=model.config.vocab_size)
correct_next_token_probs = (my_probs[:,:-1]*y_one_hot[:,1:]).sum(-1) 
my_loss=-torch.log(correct_next_token_probs).mean()
print(my_loss.item(), outputs.loss.item())

In [ ]:
with torch.no_grad():
    outputs = model(input_ids, labels=input_ids)

my_probs=F.softmax(outputs.logits, dim=-1)

sI=np.argsort(my_probs[0,-1, :].detach().cpu().float().numpy())[::-1]
for i in sI[:10]:
    print(i, round(my_probs[0, -1, i].item(),5), tokenizer.decode([i]))

In [ ]:
save_dir='/workspace/may_12_6'
os.makedirs(save_dir, exist_ok=True)
kernel_size=64
stride=64

avg_pool = torch.nn.AvgPool2d(kernel_size=kernel_size, stride=stride)
max_pool=torch.nn.MaxPool2d(kernel_size=kernel_size, stride=stride)

tensor_names=['self_attn.q_proj.weight', 'self_attn.k_proj.weight', 'self_attn.v_proj.weight', 'self_attn.o_proj.weight', 
            'mlp.gate_proj.weight', 'mlp.up_proj.weight', 'mlp.down_proj.weight']

In [ ]:
filtered_params = {name: p for name, p in model.named_parameters() if p.requires_grad}

In [ ]:
global_min=0
global_max=0
for k,v in filtered_params.items(): #Hmm i guess I'm doing the global weight average not the global grad average - do we care right now?
    if v.max().item()>global_max:
        global_max=v.max().item()
    if v.min().item()<global_min:
        global_min=v.min().item()

In [ ]:
global_max

In [ ]:
global_min

In [ ]:
out=model(input_ids.to(device), labels=input_ids.to(device))
out.loss.backward()

In [ ]:
fig=plt.figure(0,(16,9), facecolor='k')
for layer_num in range(16):
    for tensor_index in range(len(tensor_names)):
        tensor_name=tensor_names[tensor_index]
        # w=filtered_params['model.layers.'+str(layer_num)+'.'+tensor_name].detach().cpu()
        g=filtered_params['model.layers.'+str(layer_num)+'.'+tensor_name].grad.detach().cpu()
        
        # w_pooled=max_pool(w.unsqueeze(0))
        g_pooled=max_pool(g.unsqueeze(0))
        # g_pooled=avg_pool(g.unsqueeze(0))

        fig.add_subplot(7, 16, tensor_index*16+layer_num+1)    
        plt.imshow(g_pooled[0], vmin=-0.025, vmax=0.025)
        plt.axis('off')
# plt.savefig(save_dir + '/' + 'grads_max_pooled_global_norm_0001' + '.png', dpi=150, facecolor='k')

- I guess we also dont' have to really even train necessarily right? Like we could just viz gradients for various training examples? I'll want ot train at some point -> but I don't have to to gets some idea of what's going on here I think is the idea. 

In [ ]:
lr=1e-6
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [ ]:
for i in tqdm(range(10)):
    model.train()
    optimizer.zero_grad()
    outputs = model(**inputs, labels=inputs['input_ids'])
    loss = outputs.loss #Ok not just paris loss here -> not sure how much I'm worried about that
    loss.backward()
    print(loss.item())
    optimizer.step()
    
    # Add gradient clipping
    # torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

    plt.clf()
    fig=plt.figure(0,(16,9), facecolor='k')
    for layer_num in range(16):
        for tensor_index in range(len(tensor_names)):
            tensor_name=tensor_names[tensor_index]
            # w=filtered_params['model.layers.'+str(layer_num)+'.'+tensor_name].detach().cpu()
            g=filtered_params['model.layers.'+str(layer_num)+'.'+tensor_name].grad.detach().cpu()
            
            # w_pooled=max_pool(w.unsqueeze(0))
            g_pooled=max_pool(g.unsqueeze(0))
            # g_pooled=avg_pool(g.unsqueeze(0))
    
            fig.add_subplot(7, 16, tensor_index*16+layer_num+1)    
            plt.imshow(g_pooled[0], vmin=-0.025, vmax=0.025)
            plt.axis('off')
    plt.savefig(save_dir + '/' + 'grads_max_pooled_global_norm_' + str(i).zfill(3) +'.png', dpi=150, facecolor='k')

In [ ]:
model.config

In [ ]:
out=model(input_ids.to(device), labels=input_ids.to(device))
out.loss.backward()

In [ ]:
filtered_params = {name: p for name, p in model.named_parameters() if p.requires_grad}

In [ ]:
lr=1e-6
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

In [ ]:
for i in range(10):
    model.train()
    optimizer.zero_grad()
    outputs = model(**inputs, labels=inputs['input_ids'])
    loss = outputs.loss #Ok not just paris loss here -> not sure how much I'm worried about that
    loss.backward()
    print(loss.item())
    
    # Add gradient clipping
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    
    optimizer.step()

In [ ]:
with torch.no_grad():
    outputs = model(input_ids, labels=input_ids)

my_probs=F.softmax(outputs.logits, dim=-1)

sI=np.argsort(my_probs[0,5, :].detach().cpu().float().numpy())[::-1]
for i in sI[:10]:
    print(i, round(my_probs[0, 5, i].item(),5), tokenizer.decode([i]))

- Ok yeah so my poor little 3090 just can't do it -> that's a bummer.
- For some reason I do feel like I want to stick with Llama 1b -> os let's stick with llama 1b

In [ ]:
save_dir='may_12_3'
os.makedirs(save_dir, exist_ok=True)
kernel_size=64
stride=64

avg_pool = torch.nn.AvgPool2d(kernel_size=kernel_size, stride=stride)
max_pool=torch.nn.MaxPool2d(kernel_size=kernel_size, stride=stride)

tensor_names=['self_attn.q_proj.weight', 'self_attn.k_proj.weight', 'self_attn.v_proj.weight', 'self_attn.o_proj.weight', 
            'mlp.gate_proj.weight', 'mlp.up_proj.weight', 'mlp.down_proj.weight']

In [ ]:
fig=plt.figure(0,(16,9), facecolor='k')
for layer_num in range(16):
    for tensor_index in range(len(tensor_names)):
        tensor_name=tensor_names[tensor_index]
        # w=filtered_params['model.layers.'+str(layer_num)+'.'+tensor_name].detach().cpu()
        g=filtered_params['model.layers.'+str(layer_num)+'.'+tensor_name].grad.detach().cpu()
        
        # w_pooled=max_pool(w.unsqueeze(0))
        g_pooled=max_pool(g.unsqueeze(0))
        # g_pooled=avg_pool(g.unsqueeze(0))

        fig.add_subplot(7, 16, tensor_index*16+layer_num+1)    
        plt.imshow(g_pooled[0], vmin=global_min*0.0001, vmax=global_max*0.0001)
        plt.axis('off')
plt.savefig(save_dir + '/' + 'grads_max_pooled_global_norm_0001' + '.png', dpi=150, facecolor='k')

In [ ]:
global_min=0
global_max=0
for k,v in filtered_params.items(): #Hmm i guess I'm doing the global weight average not the global grad average - do we care right now?
    if v.max().item()>global_max:
        global_max=v.max().item()
    if v.min().item()<global_min:
        global_min=v.min().item()

In [ ]:
fig=plt.figure(0,(16,9), facecolor='k')
for layer_num in range(16):
    for tensor_index in range(len(tensor_names)):
        tensor_name=tensor_names[tensor_index]
        # w=filtered_params['model.layers.'+str(layer_num)+'.'+tensor_name].detach().cpu()
        g=filtered_params['model.layers.'+str(layer_num)+'.'+tensor_name].grad.detach().cpu()
        
        # w_pooled=max_pool(w.unsqueeze(0))
        g_pooled=max_pool(g.unsqueeze(0))
        # g_pooled=avg_pool(g.unsqueeze(0))

        fig.add_subplot(7, 16, tensor_index*16+layer_num+1)    
        plt.imshow(g_pooled[0], vmin=global_min*0.0001, vmax=global_max*0.0001)
        plt.axis('off')
plt.savefig(save_dir + '/' + 'grads_max_pooled_global_norm_0001' + '.png', dpi=150, facecolor='k')